In [ ]:
import pandas as pd

In [ ]:
cars = pd.read_csv("data/car_0.csv")

likes_list = [pd.read_csv("data/like_0.csv"), pd.read_csv("data/like_2.csv"), pd.read_csv("data/like_3.csv"), pd.read_csv("data/like_4.csv"),]
users_list = [pd.read_csv("data/user_0.csv"), pd.read_csv("data/user_2.csv"), pd.read_csv("data/user_3.csv"), pd.read_csv("data/user_4.csv"),]





like = pd.concat(likes_list, axis=0, ignore_index=True)
user = pd.concat(users_list, axis=0, ignore_index=True)

## Idea

Combining session based (sequential recommendations) with car data (representation)
We represent the cars as a vector of size 32
the user's ideal car (propagating vector in the RNN) is set to the users demands initially

In [ ]:
cars.head()

,brand,model,dealer,url,price,year,km,fuel,gearbox,color,doors,seats,fiscalPower,DINPower,critAir,nb_img,link,id
0,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2221069564.htm,12890,2017,60933,Essence,Manuelle,Blanc,5,5.0,4,82,1.0,7,NaN,2
1,RENAULT,Talisman,sofibrie,https://www.leboncoin.fr/voitures/2218598351.htm,24900,2019,63510,Essence,Automatique,Gris,5,5.0,8,159,1.0,20,NaN,3
2,RENAULT,Kadjar,sofibrie,https://www.leboncoin.fr/voitures/2229908445.htm,20890,2019,58858,Diesel,Manuelle,Gris,5,5.0,6,116,2.0,20,NaN,4
3,RENAULT,Koleos,sofibrie,https://www.leboncoin.fr/voitures/2301949775.htm,36490,2022,64,Essence,Automatique,Noir,5,5.0,8,158,1.0,20,NaN,5
4,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2274683789.htm,14890,2021,42555,Essence,Manuelle,Gris,5,5.0,4,83,1.0,20,NaN,6


In [ ]:
vectorized_cars = {}

In [ ]:
parameter_weights = {"brand":2, "model":2,"price":4, "year": 2, "km": 3, "fuel": 5, "gearbox":4, "color": 3, "doors": 1, "seats": 1, "fiscalPower": 2, "DINPower": 2, "critAir": 1}

In [ ]:
print("This number has to be exactly equal to 32 :")
sum(parameter_weights.values())

This number has to be exactly equal to 32 :


32

In [ ]:
def map_to_num(dataf, colname):
    mapping = {item:i for i, item in enumerate(dataf[colname].unique())}
    dataf[colname+"_n"] = dataf[colname].apply(lambda x: mapping[x])
    return mapping

In [ ]:
textual_data = ["brand", "model", "fuel", "gearbox", "color"]

In [ ]:
mapping_dict = {}

In [ ]:
for param in textual_data:
  mapping_dict[param] = map_to_num(cars, param)

Note for the future, since color takes 3 places, we can actually just use the RGB, maybe get it from the image(majority color)

In [ ]:
cars.head()

,brand,model,dealer,url,price,year,km,fuel,gearbox,color,...,DINPower,critAir,nb_img,link,id,brand_n,model_n,fuel_n,gearbox_n,color_n
0,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2221069564.htm,12890,2017,60933,Essence,Manuelle,Blanc,...,82,1.0,7,NaN,2,0,0,0,0,0
1,RENAULT,Talisman,sofibrie,https://www.leboncoin.fr/voitures/2218598351.htm,24900,2019,63510,Essence,Automatique,Gris,...,159,1.0,20,NaN,3,1,1,0,1,1
2,RENAULT,Kadjar,sofibrie,https://www.leboncoin.fr/voitures/2229908445.htm,20890,2019,58858,Diesel,Manuelle,Gris,...,116,2.0,20,NaN,4,1,2,1,0,1
3,RENAULT,Koleos,sofibrie,https://www.leboncoin.fr/voitures/2301949775.htm,36490,2022,64,Essence,Automatique,Noir,...,158,1.0,20,NaN,5,1,3,0,1,2
4,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2274683789.htm,14890,2021,42555,Essence,Manuelle,Gris,...,83,1.0,20,NaN,6,0,0,0,0,1


In [ ]:
cars["price"] = cars["price"] / 1000

In [ ]:
cars["km"] = cars["km"] / 10000
cars["year"] = cars["year"] - 1900

In [ ]:
cars.head()

,brand,model,dealer,url,price,year,km,fuel,gearbox,color,...,DINPower,critAir,nb_img,link,id,brand_n,model_n,fuel_n,gearbox_n,color_n
0,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2221069564.htm,12.89,117,6.0933,Essence,Manuelle,Blanc,...,82,1.0,7,NaN,2,0,0,0,0,0
1,RENAULT,Talisman,sofibrie,https://www.leboncoin.fr/voitures/2218598351.htm,24.90,119,6.3510,Essence,Automatique,Gris,...,159,1.0,20,NaN,3,1,1,0,1,1
2,RENAULT,Kadjar,sofibrie,https://www.leboncoin.fr/voitures/2229908445.htm,20.89,119,5.8858,Diesel,Manuelle,Gris,...,116,2.0,20,NaN,4,1,2,1,0,1
3,RENAULT,Koleos,sofibrie,https://www.leboncoin.fr/voitures/2301949775.htm,36.49,122,0.0064,Essence,Automatique,Noir,...,158,1.0,20,NaN,5,1,3,0,1,2
4,CITROEN,C3,sofibrie,https://www.leboncoin.fr/voitures/2274683789.htm,14.89,121,4.2555,Essence,Manuelle,Gris,...,83,1.0,20,NaN,6,0,0,0,0,1


In [ ]:
import numpy as np

In [ ]:
car_vectors = {}

In [ ]:
for index, row in cars.iterrows():
  car_vector = np.ndarray((32,))
  i = 0
  for param, weight in parameter_weights.items():
    if param in textual_data:
      car_vector[i:i+weight] = row[param+"_n"]
    else :
      car_vector[i:i+weight] = row[param]
    i += weight
  car_vectors[row["id"]] = car_vector

# Data Preparation

In [ ]:
like.head()

,user_id,car_id,value,swiped
0,57,2,-1,t
1,29,3,-1,f
2,29,4,-1,f
3,29,5,-1,f
4,29,6,-1,f


In [ ]:
user.head()

,maxPrice,essence,diesel,hybrid,elec,manual,auto,id
0,25000,t,f,t,t,t,t,57
1,-1,t,f,f,f,t,f,27
2,25000,t,f,t,t,t,t,28
3,13000,t,t,f,f,t,f,29
4,64000,t,t,f,f,f,t,30


In [ ]:
user_context_vectors = {}

In [ ]:
for index, row in user.iterrows():
  user_vector = np.array([0]*32)
  user_vector[3:3+4] = row["maxPrice"] / 1000
  user_vector[12] = int(row["essence"] == "t")
  user_vector[13] = int(row["diesel"]  == "t")
  user_vector[14] = int(row["hybrid"]  == "t")
  user_vector[15] = int(row["elec"] == "t")

  user_vector[17:19] = int(row["manual"]  == "t")
  user_vector[19:21] = int(row["auto"]  == "t")
  user_context_vectors[row["id"]] = user_vector

In [ ]:
inputs = []
outputs = []
current_context_vectors= user_context_vectors.copy()

In [ ]:
like.head()

,user_id,car_id,value,swiped
0,57,2,-1,t
1,29,3,-1,f
2,29,4,-1,f
3,29,5,-1,f
4,29,6,-1,f


In [ ]:
# Get rows with value 0
value_zero_rows = like[like['value'] == 0]

# Create a dictionary to store the first ideal car for each user
first_ideal_cars = value_zero_rows.groupby('user_id')['car_id'].first().to_dict()

# Define a function to get the user context
def get_user_context(user_id, current_context_vectors):
    if user_id not in current_context_vectors:
        current_context_vectors[user_id] = np.zeros(32)
    return current_context_vectors[user_id]

# Define a function to update the user context
def update_user_context(row, current_context_vectors):
    if row['value'] == 0:
        current_context_vectors[row['user_id']] = car_vectors[row['car_id']]

# Apply the update_user_context function to each row
like.apply(lambda row: update_user_context(row, current_context_vectors), axis=1)

# Iterate over the rows in 'like'
for _, row in like.iterrows():
    user_id = row['user_id']
    if user_id in first_ideal_cars:
        ideal_car_id = first_ideal_cars[user_id]
        user_context = get_user_context(user_id, current_context_vectors)
        inputs.append(np.concatenate((car_vectors[row['car_id']], np.array([row['value']] * 16), user_context)))
        outputs.append(car_vectors[ideal_car_id])

In [ ]:
len(inputs) == len(outputs)

True

In [ ]:
len(inputs)

515476

In [ ]:
len(outputs[0])

32

In [ ]:
inputs = np.nan_to_num(inputs)

In [ ]:
outputs = np.nan_to_num(outputs)

# Building the neural network

In [ ]:
import tensorflow as tf

In [ ]:

model = tf.keras.Sequential(
    [
        tf.keras.Input(shape=(64+16)),
        tf.keras.layers.Dense(128, activation="relu", name="Dense1"),
        tf.keras.layers.Dropout(0.5),  # Add dropout layer with 50% dropout rate
        tf.keras.layers.Dense(256, activation="relu", name="Dense2"),
        tf.keras.layers.Dropout(0.4),  # Add dropout layer with 50% dropout rate
        tf.keras.layers.Dense(512, activation="relu", name="UserRepresSpace"),
        tf.keras.layers.Dropout(0.3),  # Add dropout layer with 50% dropout rate
        tf.keras.layers.Dense(256, activation="relu", name="Dense3"),
        tf.keras.layers.Dropout(0.4),  # Add dropout layer with 50% dropout rate
        tf.keras.layers.Dense(128, activation="relu", name="Dense4"),
        tf.keras.layers.Dropout(0.5),  # Add dropout layer with 50% dropout rate
        tf.keras.layers.Dense(32, activation="relu", name="DenseOutput"),
    ]
)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense1 (Dense)              (None, 128)               10368     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 Dense2 (Dense)              (None, 256)               33024     
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 UserRepresSpace (Dense)     (None, 512)               131584    
                                                                 
 dropout_2 (Dropout)         (None, 512)               0         
                                                                 
 Dense3 (Dense)              (None, 256)              

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),  # Optimizer
    # Loss function to minimize
    loss=tf.keras.losses.CosineSimilarity(),
    # List of metrics to monitor
    metrics=[tf.keras.metrics.Precision()],
)

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((inputs[:int(len(inputs)*3/4)],outputs[:int(len(outputs)*3/4)]))
BATCH_SIZE = 64
SHUFFLE_BUFFER_SIZE = 100
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
test_dataset =  tf.data.Dataset.from_tensor_slices((inputs[int(len(inputs)*3/4):],outputs[int(len(outputs)*3/4):]))

In [ ]:
model.fit(train_dataset, batch_size=64, epochs=11)

Epoch 1/11
6041/6041 [==============================] - 25s 4ms/step - loss: -0.9910 - precision_1: 0.9791
Epoch 2/11
6041/6041 [==============================] - 22s 4ms/step - loss: -0.9914 - precision_1: 0.9839
Epoch 3/11
6041/6041 [==============================] - 23s 4ms/step - loss: -0.9926 - precision_1: 0.9834
Epoch 4/11
6041/6041 [==============================] - 22s 4ms/step - loss: -0.9930 - precision_1: 0.9826
Epoch 5/11
6041/6041 [==============================] - 22s 4ms/step - loss: -0.9929 - precision_1: 0.9808
Epoch 6/11
6041/6041 [==============================] - 22s 4ms/step - loss: -0.9930 - precision_1: 0.9770
Epoch 7/11
6041/6041 [==============================] - 22s 4ms/step - loss: -0.9930 - precision_1: 0.9749
Epoch 8/11
6041/6041 [==============================] - 22s 4ms/step - loss: -0.9930 - precision_1: 0.9776
Epoch 9/11
6041/6041 [==============================] - 22s 4ms/step - loss: -0.9930 - precision_1: 0.9767
Epoch 10/11
6041/6041 [==============

In [ ]:
model.save("base_model")

In [ ]:
model.evaluate(test_dataset.batch(BATCH_SIZE))

2014/2014 [==============================] - 4s 2ms/step - loss: -0.9984 - precision_1: 1.0000


[-0.9984337091445923, 1.0]

In [ ]:
import shutil
shutil.make_archive("base_model2", 'zip', "base_model")

'/content/base_model2.zip'